In [1]:
cd /Users/Liangqiaohao/Downloads/PV Lab/2020/March/Learning project/ARES

/Users/liangqiaohao/Downloads/PV Lab/2020/March/Learning project/ARES


In [2]:
import numpy as np
import torch
import math
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import GPyOpt
import GPy

import os

import matplotlib as mpl
import matplotlib.tri as tri

import ternary
import pickle
import datetime

from collections import Counter
import matplotlib.ticker as ticker

import pyDOE
import random
from scipy.stats import norm

from sklearn.ensemble import RandomForestRegressor

In [3]:
dataset = pd.read_csv(r'/Users/Liangqiaohao/Downloads/PV Lab/2020/March/Learning project/ARES/(Use this)2019-11-21.csv')
X = dataset.iloc[:,0:5]
X

,Prime Delay,Print Speed,X Offset Correction,Y Offset Correction,Score
0,0.000000,1.000000,0.000000,0.000000,0.339554
1,2.500000,3.000000,0.100000,0.100000,0.000000
2,5.000000,5.000000,-0.100000,-0.100000,0.218576
3,0.000000,0.999931,0.000006,0.000025,0.368919
4,0.000000,5.070133,0.225151,1.000000,0.000000
...,...,...,...,...,...
95,0.742365,1.714610,-0.380317,-0.416497,0.829756
96,0.000000,1.585934,-0.337143,-0.372811,0.922120
97,0.000000,1.568030,-0.335316,-0.365804,0.901296
98,0.000000,1.558826,-0.349834,-0.328645,0.936549


In [4]:
# i put negative sign here because default BO is for minimization
X['Score'] = -dataset.iloc[:,4].values


In [5]:
X

,Prime Delay,Print Speed,X Offset Correction,Y Offset Correction,Score
0,0.000000,1.000000,0.000000,0.000000,-0.339554
1,2.500000,3.000000,0.100000,0.100000,-0.000000
2,5.000000,5.000000,-0.100000,-0.100000,-0.218576
3,0.000000,0.999931,0.000006,0.000025,-0.368919
4,0.000000,5.070133,0.225151,1.000000,-0.000000
...,...,...,...,...,...
95,0.742365,1.714610,-0.380317,-0.416497,-0.829756
96,0.000000,1.585934,-0.337143,-0.372811,-0.922120
97,0.000000,1.568030,-0.335316,-0.365804,-0.901296
98,0.000000,1.558826,-0.349834,-0.328645,-0.936549


In [6]:
X_run = X.groupby(['Prime Delay', 'Print Speed', 'X Offset Correction', 'Y Offset Correction'])['Score'].agg(lambda x: x.unique().mean())
X_run = (X_run.to_frame()).reset_index()
X_run

,Prime Delay,Print Speed,X Offset Correction,Y Offset Correction,Score
0,0.0,0.1,-0.837284,-1.000000,-0.138480
1,0.0,0.1,-0.562308,-1.000000,-0.553860
2,0.0,0.1,-0.399772,1.000000,-0.237950
3,0.0,0.1,-0.383307,-0.025953,-0.529015
4,0.0,0.1,-0.263441,-1.000000,-0.444846
...,...,...,...,...,...
95,5.0,10.0,-0.453751,-1.000000,-0.649364
96,5.0,10.0,-0.392564,-1.000000,-0.641716
97,5.0,10.0,-0.030374,-1.000000,-0.394429
98,5.0,10.0,1.000000,-1.000000,0.000000


In [7]:
features = ['Prime Delay', 'Print Speed', 'X Offset Correction', 'Y Offset Correction']
X_feature = X_run[['Prime Delay', 'Print Speed', 'X Offset Correction', 'Y Offset Correction']].values

y = np.array(X_run['Score'].values)

raw_mean = np.mean(y)
raw_std = np.std(y)

In [8]:
y = (y - raw_mean) / raw_std

In [9]:
n_initial = 2
n_top = int(math.ceil(len(y) * 0.05))

top_indices = list(X_run.sort_values('Score').head(n_top).index)

# seed_list = [74,28,861,1526,2,151,19617,7,33,47302,66,552,671,25,1368,850,71,2148,2621,26, 6, 294, 9256, 851, 666]
# 25

seed_list = [5, 892, 91, 2345, 391, 77, 223, 8258, 16, 3, 84,7646,6207,4530,8206,6260,2342,3112,5187,2737,8979,4018,8223,1678,9481,2232,
             461,7694,6091, 3189,2480,9106,1316,8500,2531,5329,9872,1388,8805,2726,119,3416,7747,3927,138,1488,232,4564,3492,9841]
# 50

In [10]:
def EI(X, model, y_best):
#     xi = 0.01
    X = X.reshape([1,X_feature.shape[1]])
    
    mean, std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]

    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)

def LCB(X, model):
    X = X.reshape([1,X_feature.shape[1]])
    
    mean, std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    return 1 * std - 1 * mean

def MPI(X, model, y_best):
    X = X.reshape([1,X_feature.shape[1]])
    
    mean, std = model.predict(X)[0][0][0], model.predict(X)[1][0][0]
    
    z = (y_best - mean)/std
    return norm.cdf(z)

def calc_MAE(test_index, SL_model):
    SL_mae = 0
    
    for index in test_index:
        X_test = X_feature[index]
        y_test = y[index]
        
        X_test = X_test.reshape([1,X_feature.shape[1]])
        
        SL_mean, SL_std = SL_model.predict(X_test)[0][0][0], SL_model.predict(X_test)[1][0][0]
        
        sl_mae = (y_test - SL_mean)**2 
        SL_mae += sl_mae       
    
    return SL_mae / len(test_index)

def TopPercent(top_indices, index_):
    how_many = 0
    for i in index_:
        if i in top_indices:
            how_many += 1
    return how_many
 


In [11]:
ARD_ = False

Bias_kernel = GPy.kern.Bias(X_feature.shape[1], variance=1.)

Matern52_kernel = GPy.kern.Matern52(X_feature.shape[1], variance=1., ARD=ARD_) + Bias_kernel

In [ ]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
           

    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        GP_learn = GPy.models.GPRegression(X = np.array(X_), 
                                           Y = np.array([[i] for i in y_]), 
                                           kernel= Matern52_kernel,
                                           noise_var = 0.01
                                          )

        GP_learn.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        mae_ = calc_MAE(index_learn, GP_learn)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = EI(X_j, GP_learn, y_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_GP_EI_master', master)
   

initializing seed = 0


Process ForkPoolWorker-1196:
Process ForkPoolWorker-1199:
Process ForkPoolWorker-1197:
Process ForkPoolWorker-1200:
Process ForkPoolWorker-1198:
Process ForkPoolWorker-1195:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/liangqiaohao/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/liangqiaohao/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/liangqiaohao/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/liangqiaohao/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/Users/liangqiaohao/opt/anaconda3/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*s

Ctrl+c received, terminating and joining pool.
Warning - optimization restart 1/10 failed
Warning - optimization restart 2/10 failed
Warning - optimization restart 3/10 failed
Warning - optimization restart 4/10 failed
Warning - optimization restart 5/10 failed
Warning - optimization restart 6/10 failed
Warning - optimization restart 7/10 failed
Warning - optimization restart 8/10 failed
Warning - optimization restart 9/10 failed
Warning - optimization restart 10/10 failed


In [ ]:
########################################################################################################################################################################

In [17]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
           

    
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        GP_learn = GPy.models.GPRegression(X = np.array(X_), 
                                           Y = np.array([[i] for i in y_]), 
                                           kernel= Matern52_kernel,
                                           noise_var = 0.01
                                          )

        GP_learn.optimize_restarts(num_restarts=10,
                                   parallel = True,
                                   robust = True,
                                   optimizer = 'bfgs',
                                   max_iters=100,
                                   verbose = False)
        
        mae_ = calc_MAE(index_learn, GP_learn)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, GP_learn)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)        


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_GP_LCB11_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [ ]:
########################################################################################################################################################################

In [35]:
n_est = 50

In [37]:
def EI(X, RF_model, y_best):

    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]
    
    z = (y_best - mean)/std
    return (y_best - mean) * norm.cdf(z) + std * norm.pdf(z)



def MPI(X, RF_model, y_best):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]
    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    z = (y_best - mean)/std
    return norm.cdf(z)


def calc_MAE(test_index, RF_model):
    RF_mae = 0
    for index in test_index:
        X_test = X_feature[index]
        y_test = y[index]

        tree_predictions = []
        for j in np.arange(n_est):
            tree_predictions.append((RF_model.estimators_[j].predict(np.array([X_test]))).tolist())
        mean = np.mean(np.array(tree_predictions), axis=0)[0]

        std = np.std(np.array(tree_predictions), axis=0)[0]

        mae_ = (y_test - mean)**2 
        RF_mae += mae_
        
    return RF_mae / len(test_index)

In [25]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
                
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        mae_ = calc_MAE(index_learn, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = EI(X_j, RF_model, y_best)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_EI_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [ ]:
########################################################################################################################################################################

In [ ]:
def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 1 * std - 1 * mean

In [38]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
                
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        mae_ = calc_MAE(index_learn, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_LCB11_master', master)

initializing seed = 0
Finished seed
initializing seed = 1
Finished seed
initializing seed = 2
Finished seed
initializing seed = 3
Finished seed
initializing seed = 4
Finished seed
initializing seed = 5
Finished seed
initializing seed = 6
Finished seed
initializing seed = 7
Finished seed
initializing seed = 8
Finished seed
initializing seed = 9
Finished seed
initializing seed = 10
Finished seed
initializing seed = 11
Finished seed
initializing seed = 12
Finished seed
initializing seed = 13
Finished seed
initializing seed = 14
Finished seed
initializing seed = 15
Finished seed
initializing seed = 16
Finished seed
initializing seed = 17
Finished seed
initializing seed = 18
Finished seed
initializing seed = 19
Finished seed
initializing seed = 20
Finished seed
initializing seed = 21
Finished seed
initializing seed = 22
Finished seed
initializing seed = 23
Finished seed
initializing seed = 24
Finished seed
initializing seed = 25
Finished seed
initializing seed = 26
Finished seed
initializin

In [ ]:
########################################################################################################################################################################

In [ ]:
def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 1 * std - 0 * mean

In [ ]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
                
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        mae_ = calc_MAE(index_learn, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_LCB10_master', master)

In [22]:
########################################################################################################################################################################

In [ ]:
def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 0 * std - 1 * mean

In [ ]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
                
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        mae_ = calc_MAE(index_learn, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_LCB01_master', master)

In [ ]:
########################################################################################################################################################################

In [ ]:
def LCB(X, RF_model):
    
    tree_predictions = []
    for j in np.arange(n_est):
        tree_predictions.append((RF_model.estimators_[j].predict(np.array([X]))).tolist())
    mean = np.mean(np.array(tree_predictions), axis=0)[0]

    
    std = np.std(np.array(tree_predictions), axis=0)[0]

    
    return 75 * std - 25 * mean

In [ ]:
index_collection = []
X_collection = []
y_collection = []
TopPercent_collection = []
MAE_collection = []


for s in seed_list:
    print('initializing seed = ' +str(seed_list.index(s)))
    random.seed(s)
    
    indices = list(np.arange(len(X_run)))
    
    
    X_top = []
    y_top = []
    for i in top_indices:
        X_top.append(X_feature[i])
        y_top.append(y[i])
        
    index_learn = indices.copy()
    
    index_ = random.sample(index_learn, n_initial)
    X_ = []
    y_ = []
    TopPercent_ = []
    MAE_ = []
    for i in index_:
        X_.append(X_feature[i])
        y_.append(y[i])
        index_learn.remove(i)
                
    for i in np.arange(len(index_learn)):
        
        y_best = np.min(y_)
        
        RF_model = RandomForestRegressor(n_estimators= n_est)
        RF_model.fit(X_, y_)
        
        mae_ = calc_MAE(index_learn, RF_model)
        MAE_.append(mae_)
        TopPercent_.append(TopPercent(top_indices, index_))
        
        next_index = None
        max_ac = -10**10
        for j in index_learn:
            X_j = X_feature[j]
            y_j = y[j]
            
            ac_value = LCB(X_j, RF_model)
            
            if max_ac <= ac_value:
                max_ac = ac_value
                next_index = j
                
        X_.append(X_feature[next_index])
        y_.append(y[next_index])
        index_learn.remove(next_index)
        index_.append(next_index)


    index_collection.append(index_)
    X_collection.append(X_)
    y_collection.append(y_)
    TopPercent_collection.append(TopPercent_)
    MAE_collection.append(MAE_)
    
    print('Finished seed')
    
    
master = np.array([index_collection, X_collection, y_collection, TopPercent_collection, MAE_collection])
np.save('STANDARD_RF_LCB7525_master', master)